In [1]:
import numpy as np
import os
import json

In [1]:
from filtool_script import FiltoolExec

In [2]:
fil_exec = FiltoolExec('/home/rsenzel/Code/inception/dev_injector/fb/controlfb2_uhf.fil', 'pars.json', 'test_dir', 1)

In [4]:
fil_exec.processing_args

{'rfi_flags': 'kadaneF 8 4',
 'temp_filesystem': '/beegfs/',
 'fscrunch': 1,
 'snr_threshold': 8.5,
 'nharmonics': '4',
 'fft_length': 8388608,
 'start_accel': -25,
 'end_accel': 25,
 'accel_tol': 1.11,
 'candidate_limit': 1000,
 'ddplan': '0.000     517.800    0.30 1    1726 0.744\r\n517.800   834.800    0.50 2     634 0.1366\r\n834.800  1589.800    1.00 4     755 0.08136\r\n1589.800 3001.800    2.00 8     706 0.03804',
 'overrides': 'both',
 'birdie_list': '1000.00:0.05,1500.00:0.05,1300.00:0.05',
 'channel_mask': '856.0:863.94140625,935.4140625:940.84765625',
 'ram_limit': 24}

In [3]:
ddplan = list(fil_exec.create_DDplan())
filplan = fil_exec.create_filplan(ddplan)

In [5]:
filplan

'test_dir/filplan.json'

In [ ]:
f'overview_dm_0.000000_103.800000.xml'

In [ ]:
xml_file_names = [f'overview_dm_{dm_range.low:.6f}_{dm_range.high:.6f}.xml' for dm_range in ddplan]

In [6]:
for i in ddplan:
    print(i)

DMRange(low_dm=0.0, high_dm=517.8, dm_step=0.3, tscrunch=1.0)
DMRange(low_dm=517.8, high_dm=834.8, dm_step=0.5, tscrunch=2.0)
DMRange(low_dm=834.8, high_dm=1589.8, dm_step=1.0, tscrunch=4.0)
DMRange(low_dm=1589.8, high_dm=3001.8, dm_step=2.0, tscrunch=8.0)


In [45]:
dm_range = np.linspace(ddplan[0].low_dm, ddplan[0].high_dm, int((ddplan[0].high_dm-ddplan[0].low_dm)/ddplan[0].dm_step), endpoint=False)
dm1 = np.linspace(ddplan[1].low_dm, ddplan[1].high_dm, int((ddplan[1].high_dm-ddplan[1].low_dm)/ddplan[1].dm_step), endpoint=False)

In [82]:
dm_trials_list = []
tscrunch_list = []
for i, dm_range in enumerate(ddplan):
    n_trial = int((dm_range.high_dm-dm_range.low_dm)/dm_range.dm_step)
    n_trial, endpoint = (n_trial + 1, True) if i == len(ddplan)-1 else (n_trial, False)
    dm_list_i = np.linspace(dm_range.low_dm, dm_range.high_dm, n_trial, endpoint=endpoint)
    tscrunch = np.ones_like(dm_list_i) * dm_range.tscrunch
    dm_trials_list.append(dm_list_i)
    tscrunch_list.append(tscrunch)

In [83]:
dm_trials = np.concat(dm_trials_list)
tscrunch = np.concat(tscrunch_list)
tscrunch

array([1., 1., 1., ..., 8., 8., 8.])

In [66]:
def find_n_nearest(dm_arr, tscrunch_arr, target, n):
    nearest_indices = np.argsort(np.abs(dm_arr - target))[:n]
    dm_values = dm_arr[nearest_indices]
    tscrunch_values = tscrunch_arr[nearest_indices]
    return dm_values, tscrunch_values

In [71]:
a, b = find_n_nearest(dm_trials, tscrunch, 517.7, 5)
a, b

(array([517.8, 517.5, 517.2, 518.3, 516.9]), array([2., 1., 1., 2., 1.]))

In [81]:
np.where(np.array([dm_range.tscrunch for dm_range in ddplan] )== 2)[0][0]

np.int64(1)

In [73]:
a[np.where(b == 1)]

array([517.5, 517.2, 516.9])

In [50]:
my_dms = np.random.uniform(50, 1500, size=10)
nearest = 5
dm_search = np.zeros((len(my_dms), nearest))
my_dms, dm_search

(array([1072.13331736, 1374.67901752, 1121.25312234, 1487.44617915,
        1061.48080809, 1298.42985136, 1143.93419058,  585.98544057,
         226.25915701,  530.1386847 ]),
 array([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]]))